In [1]:
%reload_kedro

2023-05-09 14:21:59,938 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: /Users/gpalazzo/Desktop/dev/crypto_msc_thesis
2023-05-09 14:22:00,790 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-05-09 14:22:01,111 - kedro.extras.extensions.ipython - INFO - ** Kedro project crypto_thesis
2023-05-09 14:22:01,112 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


In [2]:
import numpy as np

In [3]:
df_spine = catalog.load("spine_labeled")
df_mt_multic = catalog.load("master_table_multic")
df_window_nbr = catalog.load("window_nbr_lookup_multic")

2023-05-09 14:22:01,125 - kedro.io.data_catalog - INFO - Loading data from `spine_labeled` (ParquetDataSet)...
2023-05-09 14:22:01,556 - kedro.io.data_catalog - INFO - Loading data from `master_table_multic` (ParquetDataSet)...
2023-05-09 14:22:01,565 - kedro.io.data_catalog - INFO - Loading data from `window_nbr_lookup_multic` (ParquetDataSet)...


In [4]:
df_mt = df_mt_multic[["window_nbr"]] \
                    .merge(df_window_nbr, on="window_nbr", how="inner")
assert df_mt.shape[0] == df_mt_multic.shape[0]

In [5]:
df_mt.head(2)

,window_nbr,open_time,close_time,target_time
0,1,2019-11-28 08:30:00,2019-11-29 05:45:00,2019-11-29 21:45:00
1,2,2019-11-29 06:00:00,2019-11-29 21:45:00,2019-12-01 00:15:00


In [6]:
df = df_mt.merge(df_spine[["open_time", "close_time", "target_time", "logret_cumsum"]],
                on=["open_time", "close_time", "target_time"],
                how="inner")
assert df.shape[0] == df_mt.shape[0]

In [7]:
df.head()

,window_nbr,open_time,close_time,target_time,logret_cumsum
0,1,2019-11-28 08:30:00,2019-11-29 05:45:00,2019-11-29 21:45:00,-0.001806
1,2,2019-11-29 06:00:00,2019-11-29 21:45:00,2019-12-01 00:15:00,0.030120
2,3,2019-11-29 22:00:00,2019-12-01 00:15:00,2019-12-01 17:45:00,-0.026480
3,4,2019-12-01 00:30:00,2019-12-01 17:45:00,2019-12-02 17:30:00,-0.022385
4,5,2019-12-01 18:00:00,2019-12-02 17:30:00,2019-12-04 01:45:00,-0.003861


In [8]:
df.close_time.min(), df.close_time.max()

(Timestamp('2019-11-29 05:45:00'), Timestamp('2022-10-02 16:00:00'))

In [9]:
df.logret_cumsum.min(), df.logret_cumsum.max()

(-0.21988566074418078, 0.3242773540806224)

In [10]:
df.loc[:, "pctchg_cumsum"] = np.exp(df["logret_cumsum"]) - 1
df.pctchg_cumsum.min(), df.pctchg_cumsum.max()

(-0.1973894373893308, 0.38303084317735747)